In [2]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install haversine

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [7]:
import folium
from haversine import haversine, Unit
from geopy.geocoders import Nominatim

import numpy as np
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import re

%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [10, 6]

In [8]:
stay = pd.read_csv('data/서울교통공사_1-8호선 지하철역 위경도 좌표정보_20220621.csv', encoding='cp949')
stay.index = stay['역명']
del stay['역명'] # del을 이용해서 하나씩 삭제
del stay['데이터기준일자']
stay

,연번,호선,역번호,경도,위도
역명,,,,,
서울,1,1,150,126.972533,37.553150
시청,2,1,151,126.975407,37.563590
종각,3,1,152,126.983116,37.570203
종로3가,4,1,153,126.992095,37.570429
종로5가,5,1,154,127.001900,37.570971
...,...,...,...,...,...
남한산성입구,272,8,2823,127.159845,37.451568
단대오거리,273,8,2824,127.156735,37.445057
신흥,274,8,2825,127.147590,37.440952


In [9]:
apt = pd.read_excel('data/20210801-20220731.서울시아파트(매매)_실거래가.xlsx')
apt['주소'] = apt['시군구']+' '+apt['도로명']
apt.drop(['번지','본번','부번','시군구','도로명'], axis=1, inplace=True)
apt

,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,주소
0,개포6차우성아파트1동~8동,79.97,202108,11,"200,000",3,1987,서울특별시 강남구 개포동 언주로 3
1,개포6차우성아파트1동~8동,79.97,202109,2,"220,000",2,1987,서울특별시 강남구 개포동 언주로 3
2,개포6차우성아파트1동~8동,79.97,202112,4,"215,000",3,1987,서울특별시 강남구 개포동 언주로 3
3,개포6차우성아파트1동~8동,79.97,202204,12,"220,000",4,1987,서울특별시 강남구 개포동 언주로 3
4,개포6차우성아파트1동~8동,79.97,202204,21,"220,000",2,1987,서울특별시 강남구 개포동 언주로 3
...,...,...,...,...,...,...,...,...
20491,한신아파트(103~109),59.76,202203,27,"73,000",20,1997,서울특별시 중랑구 중화동 동일로 752
20492,한영(101),84.69,202111,4,"50,000",2,2003,서울특별시 중랑구 중화동 동일로144길 74
20493,한영(101),84.69,202204,9,"49,900",7,2003,서울특별시 중랑구 중화동 동일로144길 74
20494,한영(103),104.93,202108,20,"61,500",7,2003,서울특별시 중랑구 중화동 동일로144길 74


In [10]:
address = apt['주소']
address

0              서울특별시 강남구 개포동 언주로 3
1              서울특별시 강남구 개포동 언주로 3
2              서울특별시 강남구 개포동 언주로 3
3              서울특별시 강남구 개포동 언주로 3
4              서울특별시 강남구 개포동 언주로 3
                   ...            
20491        서울특별시 중랑구 중화동 동일로 752
20492     서울특별시 중랑구 중화동 동일로144길 74
20493     서울특별시 중랑구 중화동 동일로144길 74
20494     서울특별시 중랑구 중화동 동일로144길 74
20495    서울특별시 중랑구 중화동 봉화산로27가길 19
Name: 주소, Length: 20496, dtype: object

In [11]:
geo_local = Nominatim(user_agent='South Korea') # 지역선택

def geocoding(address): 
    geo = geo_local.geocode(address)
    x_y = [geo.latitude, geo.longitude]
    return x_y

latitude = []
longitude = []
exception_addr = []

for i in address:
    try:
        latitude.append(geocoding(i)[0])
        longitude.append(geocoding(i)[1])
    except:
        exception_addr.append(i) # 위경도를 잡지 못한경우 exception에 따로 저장, 여기서는 4개 데이터가 빠지게 됨
# 시간이 좀 걸림, 약 1분 30초정도 소요됨

In [12]:
print(latitude)
print(longitude)
print(exception_addr)

[37.483371950000006, 37.483371950000006, 37.483371950000006, 37.483371950000006, 37.483371950000006, 37.483371950000006, 37.4913783, 37.4913783, 37.4913783, 37.4913783, 37.4913783, 37.4913783, 37.4913783, 37.4913783, 37.4913783, 37.4913783, 37.4913783, 37.4913783, 37.4913783, 37.4913783, 37.4913783, 37.4913783, 37.4913783, 37.4913783, 37.4968938, 37.4968938, 37.48786825, 37.48786825, 37.4884808, 37.4884808, 37.4884808, 37.4884808, 37.4884808, 37.47929375, 37.47929375, 37.47929375, 37.47929375, 37.4744964, 37.4891234, 37.4891234, 37.4891234, 37.4891234, 37.4891234, 37.4891234, 37.4891234, 37.4891234, 37.4891234, 37.4891234, 37.4891234, 37.4891234, 37.4891234, 37.4891234, 37.4891234, 37.4855881, 37.4855881, 37.4855881, 37.4855881, 37.4855881, 37.4855881, 37.4855881, 37.4855881, 37.4855881, 37.4855881, 37.4855881, 37.4855881, 37.4855881, 37.4855881, 37.4855881, 37.4855881, 37.4855881, 37.4855881, 37.4855881, 37.4855881, 37.4968938, 37.4968938, 37.4968938, 37.4968938, 37.4968938, 37.473424